In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LST:
  def __init__(self, L, f, fl, fll, default=0, ldefault=0):
    self.f = f
    self.fl = fl
    self.fll = fll
    self.default = default
    self.ldefault = ldefault
    self.len = len(L)
    self.seg = [default] * self.len + L
    self.lazy = [ldefault] * self.len
    self.cnt = [0] * self.len + [1] * self.len

    for i in reversed(range(self.len)):
      self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      self.cnt[i] = self.cnt[i*2] + self.cnt[i*2+1]

  def _push(self, i, v) :
    self.seg[i] = self.fl(self.seg[i], v, self.cnt[i])
    if i < self.len :
      self.lazy[i] = self.fll(self.lazy[i], v)

  def _apply(self, i) :
    if self.lazy[i] == self.ldefault: return
    self._push(i*2, self.lazy[i])
    self._push(i*2+1, self.lazy[i])
    self.lazy[i] = self.ldefault 

  def _propagate(self, i):
    for h in reversed(range(1, self.len.bit_length()+1)) :
      idx = i >> h

      if idx == self.default : continue
      self._apply(idx)

  def _build(self, i) :
    while i :
      if i < self.len :
        self._apply(i)
        self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      i >>= 1

  def query(self, _s, _e) : #[s, e]
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    res = self.default
    while s < e :
      if s & 1 :
        res = self.f(res, self.seg[s])
        s += 1
      if e & 1 :
        e -= 1
        res = self.f(res, self.seg[e])
      s >>= 1
      e >>= 1
    return res

  def update(self, _s, _e, x):
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    while s < e :
      if s & 1 :
        self._push(s, x)
        s += 1
      if e & 1 :
        e -= 1
        self._push(e, x)
      s >>= 1
      e >>= 1
    
    self._build(_s)
    self._build(_e)
  
MAX = 10 ** 18 + 1
def sol() :
  N = int(input())
  Q = []
  for _ in range(N) :
    Q.append(list(map(int, input().split())))
  
  P = [1, MAX]
  for _, s, e in Q :
    P.append(s)
    P.append(e)
  P = sorted(set(P))
  Pzip = {v:i for i, v in enumerate(P)}
  
  def f(a, b) :
    if a[1] == 0 and b[1] == 0: return (max(a[0], b[0]), 0) #어떤 수도 mex의 후보가 아님
    return (max(a[0] * a[1], b[0] * b[1]), 1) #두 후보중 점수가 높은 것(index가 낮은 것)을 후보로 반환
  
  def fl(v, x, _) :
    i, can = v
    if x[0]: return (i, x[1]) #flag가 켜져있으면 set
    return (i, can ^ x[1])

  def fll(lv, x) :
    v, xor = lv
    v2, xor2 = x
    if v2: return x
    return (v, xor ^ xor2)

  ans = []
  lst = LST([(i, 1) for i in reversed(range(len(P)))], f, fl, fll, (0, 0), (0, 0))
  for q, s, e in Q :
    l, r = Pzip[s], Pzip[e]
    match q: 
      case 1 :
        lst.update(l, r, (1, 0))
      case 2 :
        lst.update(l, r, (1, 1))
      case 3 :
        lst.update(l, r, (0, 1))
    
    # for i in range(len(P)) : lst._apply(i)
    # print(lst.seg)
    # ans.append(lst.query(0, len(P)-1))
    ans.append(P[len(P) - lst.query(0, len(P)-1)[0]-1])

  sys.stdout.write("\n".join(map(str, ans)))
sol()

### 아이디어
- 트라이로 접근하면 쉬울줄 알았는데 l,r의 범위가 무려 $10^{18}$ 이였다. 각각의 상태를 저장하는 방법은 아닐 것이다.
  - 여기서 해결법을 찾지 못하고 태그를 보니 lazy 세그 문제였다. 
  - 끄적이면서 구조체 lazy 발상을 했던거같은데 좌표압축을 생각해내지 못한게 아쉽다.
- lazy 구조체의 정의를 `(flag, op)` 로 구성했다. `flag`가 켜져있으면, op를 대입한다는 뜻이고, 꺼져있으면 기존의 op에 대해 $\oplus op$ 를 수행한다는 뜻이다.
- 내자마자 WA를 받았다. 분명 `f` 에서 둘다 0일때 그 index는 상관이 없어야 할 터인데, 그 값에따라 예제출력의 정답이 다르게 나오는것부터가 이상했다.
  - lazy 세그가 제대로 분할되지 않은 것 같다.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LST:
  def __init__(self, L, f, fl, fll, default=0, ldefault=0):
    self.f = f
    self.fl = fl
    self.fll = fll
    self.default = default
    self.ldefault = ldefault
    self.len = len(L)
    self.seg = [default] * self.len + L
    self.lazy = [ldefault] * self.len
    self.cnt = [0] * self.len + [1] * self.len

    for i in reversed(range(self.len)):
      self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      self.cnt[i] = self.cnt[i*2] + self.cnt[i*2+1]

  def _push(self, i, v) :
    self.seg[i] = self.fl(self.seg[i], v, self.cnt[i])
    if i < self.len :
      self.lazy[i] = self.fll(self.lazy[i], v)

  def _apply(self, i) :
    if self.lazy[i] == self.ldefault: return
    self._push(i*2, self.lazy[i])
    self._push(i*2+1, self.lazy[i])
    self.lazy[i] = self.ldefault 

  def _propagate(self, i):
    for h in reversed(range(1, self.len.bit_length()+1)) :
      idx = i >> h

      if idx == self.default : continue
      self._apply(idx)

  def _build(self, i) :
    while i :
      if i < self.len :
        self._apply(i)
        self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      i >>= 1

  def query(self, _s, _e) : #[s, e]
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    res = self.default
    while s < e :
      if s & 1 :
        res = self.f(res, self.seg[s])
        s += 1
      if e & 1 :
        e -= 1
        res = self.f(res, self.seg[e])
      s >>= 1
      e >>= 1
    return res

  def update(self, _s, _e, x):
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    while s < e :
      if s & 1 :
        self._push(s, x)
        s += 1
      if e & 1 :
        e -= 1
        self._push(e, x)
      s >>= 1
      e >>= 1
    
    self._build(_s)
    self._build(_e)
  
MAX = 10 ** 18 + 1
def sol() :
  N = int(input())
  Q = []
  for _ in range(N) :
    Q.append(list(map(int, input().split())))
  
  P = [1, MAX]
  for _, s, e in Q :
    P.append(s)
    P.append(e)
  P = sorted(set(P))
  Pzip = {v:i for i, v in enumerate(P)}
  
  def f(a, b) :
    # print(a, b, max(a[0] * a[1], b[0] * b[1]))
    if a[1] == 0 and b[1] == 0: return (0, 0) #어떤 수도 mex의 후보가 아님
    # if a[1] == 0 and b[1] == 0: return (max(a[0], b[0]), 0)
    return (max(a[0] * a[1], b[0] * b[1]), 1) #두 후보중 점수가 높은 것(index가 낮은 것)을 후보로 반환
  
  def fl(v, x, cnt) :
    i, can = v
    if x[0]: return (i, x[1]) #set flag가 켜져있으면 set
    return (i, can ^ x[1]) if cnt & 1 else (i, can)

  def fll(lv, x) :
    v, xor, pri1 = lv
    v2, xor2, pri2 = x
    if pri1 == pri2 : return lv
    if v and v2: 
      if pri1 > pri2 : return lv
      return x 
    elif v :
      if pri1 > pri2 : return lv
      return (v, xor ^ xor2, pri2) #1 x -> 0 1
    elif v2 :
      if pri1 > pri2 : return (v2, xor ^ xor2, pri1)
      return x
    return (v, xor ^ xor2, max(pri1, pri2))

  ans = []
  lst = LST([(i, 1) for i in reversed(range(len(P)))], f, fl, fll, (0, 0), (0, 0, 0))
  for i, (q, s, e) in enumerate(Q) :
    l, r = Pzip[s], Pzip[e]
    match q: 
      case 1 :
        lst.update(l, r, (1, 0, i))
      case 2 :
        lst.update(l, r, (1, 1, i))
      case 3 :
        lst.update(l, r, (0, 1, i))
    
    # for i in range(len(P)) : lst._apply(i)
    # print(lst.seg)
    # ans.append(lst.query(0, len(P)-1))
    res = lst.query(0, len(P)-1)
    assert res[1] == 1
    ans.append(P[len(P) - res[0]-1])

  sys.stdout.write("\n".join(map(str, ans)))
sol()

- 연산 순서가 문제인지 싶어서 lazy에 priority도 반영시켰지만 계속 틀린다. 뭔가 접근이 틀린거 같기도.

### 풀이
- 세그먼트 트리로 mex를 구하는 방법을 다시 알아야 할 필요가 있었다.
  - 원소의 존재 여부를 0과 1로 두고, add를 f로 둔다. 좌 -> 우 자식 순서로 살펴보면서 해당 노드의 값이 꽉찬 노드의 값과 같지 않다면 그 자식쪽에 mex가 존재한다.